based on https://www.kaggle.com/competitions/word2vec-nlp-tutorial/overview

In [39]:
!cp kaggle.json /content
!chmod 600 /content/kaggle.json

In [1]:
import numpy as np
import pandas as pd 

In [36]:
!pip install kaggle --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.13-py3-none-any.whl size=77717 sha256=3277eda9a26c5478a2d03adf2156264a881b51150cccc67ee3db33b3120e5db6
  Stored in directory: /root/.cache/pip/wheels/92/8c/37/96a1971bedc1e74057af1e4852f18de7e8286dea4f12928e6c
Successfully built kaggle


In [2]:

## set up custom path to kaggle.json: 
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

from pathlib import Path
path = Path('word2vec-nlp-tutorial')
if not path.exists():
    import zipfile,kaggle
    kaggle.api.competition_download_cli(str(path))
    # for each zip file in the path unzip it
    for f in path.glob('*.zip'):
        zipfile.ZipFile(f).extractall(path)

In [3]:
train =  pd.read_csv(path/'labeledTrainData.tsv',sep='\t')

In [4]:
train.describe(include='object')

,id,review
count,25000,25000
unique,25000,24904
top,5814_8,"When i got this movie free from my job, along ..."
freq,1,3


In [5]:
train.review[0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [6]:
from bs4 import BeautifulSoup

In [7]:
bs1 = BeautifulSoup(train.review[0])
bs1.get_text()

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 mi

In [8]:
import re 

letters_only = re.sub("[^a-zA-Z]", " ", bs1.get_text())
letters_only

'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

In [11]:
words = letters_only.lower().split()
words[:5],len(words)

(['with', 'all', 'this', 'stuff', 'going'], 437)

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from nltk.corpus import stopwords 

In [15]:
words = [w for w in words if not w in stopwords.words("english")]
words[:5],len(words)

(['stuff', 'going', 'moment', 'mj', 'started'], 219)

In [14]:
import re 
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords 

def clean_review_words(review, remove_stopwords=False):
    bs = BeautifulSoup(review)
    letters_only = re.sub("[^a-zA-Z]", " ", bs.get_text())
    words = letters_only.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return  words


def clean_review(review, remove_stopwords=False):
    words = clean_review_words(review, remove_stopwords)
    return( " ".join(words))

In [15]:
clean_review_words(train.review[0],True)[:5]

['stuff', 'going', 'moment', 'mj', 'started']

In [12]:
clean_review(train.review[0])

'with all this stuff going down at the moment with mj i ve started listening to his music watching the odd documentary here and there watched the wiz and watched moonwalker again maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent moonwalker is part biography part feature film which i remember going to see at the cinema when it was originally released some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for minutes or s

In [18]:
train['clean_review'] = train.review.apply(clean_review)

/usr/local/lib/python3.9/dist-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [19]:
train['review'][0], train['clean_review'][0]

("With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally st

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, 
                             preprocessor = None, stop_words = None, max_features = 5000)

train_data_features = vectorizer.fit_transform(train.clean_review)
train_data_features


<25000x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 1975048 stored elements in Compressed Sparse Row format>

In [21]:
train_data_features = train_data_features.toarray()
train_data_features.shape

(25000, 5000)

In [23]:
vocab = vectorizer.get_feature_names()
vocab[:10]

['abandoned',
 'abc',
 'abilities',
 'ability',
 'able',
 'abraham',
 'absence',
 'absent',
 'absolute',
 'absolutely']

In [24]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit( train_data_features, train["sentiment"] )
forest

RandomForestClassifier()

In [25]:
test = pd.read_csv(path/'testData.tsv',sep='\t')
test['clean_review'] = test.review.apply(clean_review)
test_data_features = vectorizer.transform(test.clean_review).toarray()
test_data_features.shape


/usr/local/lib/python3.9/dist-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


(25000, 5000)

In [29]:
test_data_features[3][:50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [30]:
test.head()

,id,review,clean_review
0,12311_10,Naturally in a film who's main themes are of m...,naturally film main themes mortality nostalgia...
1,8348_2,This movie is a disaster within a disaster fil...,movie disaster within disaster film full great...
2,5828_4,"All in all, this is a movie for kids. We saw i...",movie kids saw tonight child loved one point k...
3,7186_2,Afraid of the Dark left me with the impression...,afraid dark left impression several different ...
4,12128_7,A very accurate depiction of small time mob li...,accurate depiction small time mob life filmed ...


In [31]:
test['sentiment'] = forest.predict(test_data_features)

In [42]:
test.head()

,id,review,clean_review,sentiment
0,12311_10,Naturally in a film who's main themes are of m...,naturally film main themes mortality nostalgia...,1
1,8348_2,This movie is a disaster within a disaster fil...,movie disaster within disaster film full great...,0
2,5828_4,"All in all, this is a movie for kids. We saw i...",movie kids saw tonight child loved one point k...,1
3,7186_2,Afraid of the Dark left me with the impression...,afraid dark left impression several different ...,1
4,12128_7,A very accurate depiction of small time mob li...,accurate depiction small time mob life filmed ...,1


In [33]:
test[['id','sentiment']].to_csv('submission.csv',index=False)

In [41]:
import kaggle
kaggle.api.competition_submit('submission.csv', 'Random Forest from sample notebook', 
                              'word2vec-nlp-tutorial')


100%|██████████| 227k/227k [00:00<00:00, 533kB/s]


Successfully submitted to Bag of Words Meets Bags of Popcorn

Score for the submission above is: 0.84508

lets continue with part 2 https://www.kaggle.com/competitions/word2vec-nlp-tutorial/overview/part-2-word-vectors

In [6]:
test = pd.read_csv( path/"testData.tsv", delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv(path/"unlabeledTrainData.tsv", delimiter="\t", quoting=3)

In [8]:
train.review.size, test.review.size, unlabeled_train.review.size

(25000, 25000, 50000)

In [10]:
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [16]:
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append( clean_review_words( raw_sentence, remove_stopwords))
    return sentences


In [19]:
review_to_sentences(train.review[0],tokenizer)[:3]

[['with',
  'all',
  'this',
  'stuff',
  'going',
  'down',
  'at',
  'the',
  'moment',
  'with',
  'mj',
  'i',
  've',
  'started',
  'listening',
  'to',
  'his',
  'music',
  'watching',
  'the',
  'odd',
  'documentary',
  'here',
  'and',
  'there',
  'watched',
  'the',
  'wiz',
  'and',
  'watched',
  'moonwalker',
  'again'],
 ['maybe',
  'i',
  'just',
  'want',
  'to',
  'get',
  'a',
  'certain',
  'insight',
  'into',
  'this',
  'guy',
  'who',
  'i',
  'thought',
  'was',
  'really',
  'cool',
  'in',
  'the',
  'eighties',
  'just',
  'to',
  'maybe',
  'make',
  'up',
  'my',
  'mind',
  'whether',
  'he',
  'is',
  'guilty',
  'or',
  'innocent'],
 ['moonwalker',
  'is',
  'part',
  'biography',
  'part',
  'feature',
  'film',
  'which',
  'i',
  'remember',
  'going',
  'to',
  'see',
  'at',
  'the',
  'cinema',
  'when',
  'it',
  'was',
  'originally',
  'released']]

In [20]:
sentences = []  # Initialize an empty list of sentences

for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

/usr/local/lib/python3.9/dist-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/bs4/__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


In [21]:
len(sentences)

794335

In [27]:
sentences[5]

['why', 'he', 'wants', 'mj', 'dead', 'so', 'bad', 'is', 'beyond', 'me']

In [28]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

In [30]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 44.6 MB/s eta 0:00:0000:0100:01


In [35]:
num_features = 100    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

from gensim.models import word2vec

model = word2vec.Word2Vec(sentences, workers=num_workers, min_count = min_word_count, 
                          window = context, sample = downsampling)

model.init_sims(replace=True)

model_name = "100features_40minwords_10context"
model.save(model_name)

2023-04-21 09:48:06,763 : INFO : collecting all words and their counts
2023-04-21 09:48:06,765 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-04-21 09:48:06,828 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2023-04-21 09:48:06,891 : INFO : PROGRESS: at sentence #20000, processed 451842 words, keeping 24946 word types
2023-04-21 09:48:06,958 : INFO : PROGRESS: at sentence #30000, processed 671053 words, keeping 30029 word types
2023-04-21 09:48:07,030 : INFO : PROGRESS: at sentence #40000, processed 897624 words, keeping 34348 word types
2023-04-21 09:48:07,101 : INFO : PROGRESS: at sentence #50000, processed 1120158 words, keeping 37805 word types
2023-04-21 09:48:07,186 : INFO : PROGRESS: at sentence #60000, processed 1340913 words, keeping 40769 word types
2023-04-21 09:48:07,267 : INFO : PROGRESS: at sentence #70000, processed 1564763 words, keeping 43362 word types
2023-04-21 09:48:07,345 : INFO : PROGRESS: 

In [59]:
wv = model.wv

In [52]:
wv.doesnt_match("man woman child kitchen".split())

'kitchen'

In [53]:
wv.doesnt_match("france england germany berlin".split())


'berlin'

In [54]:
wv.doesnt_match("paris berlin london austria".split())


'paris'

In [56]:
wv.most_similar("man")

[('woman', 0.730812132358551),
 ('lad', 0.706100583076477),
 ('boy', 0.6874200105667114),
 ('soldier', 0.6723482608795166),
 ('lady', 0.6720548868179321),
 ('guy', 0.6528904438018799),
 ('person', 0.634056806564331),
 ('monk', 0.6250230073928833),
 ('chap', 0.6122916340827942),
 ('farmer', 0.6015139818191528)]

In [63]:
x = wv["queen"] - wv["man"]

In [64]:
# gets a word that is similar to the vector x
wv.similar_by_vector(x)

[('complement', 0.0),
 ('fares', 0.0),
 ('thirsty', 0.0),
 ('lipstick', 0.0),
 ('humane', 0.0),
 ('highlighted', 0.0),
 ('canadians', 0.0),
 ('defence', 0.0),
 ('refund', 0.0),
 ('goo', 0.0)]

In [65]:
x = wv["man"] + wv["royal"] 

In [66]:
wv.similar_by_vector(x)# doesn't seem to work   

[('royal', 0.7769883871078491),
 ('queen', 0.7769883871078491),
 ('man', 0.7769883871078491),
 ('soldier', 0.7629314661026001),
 ('lad', 0.7098776698112488),
 ('farmer', 0.7046628594398499),
 ('ruler', 0.6472869515419006),
 ('dictator', 0.6392303705215454),
 ('squire', 0.6382250785827637),
 ('priest', 0.6337595582008362)]

In [68]:
model.wv.distance()

<bound method KeyedVectors.distance of <gensim.models.keyedvectors.KeyedVectors object at 0x7f89a0a9a160>>

transformers as per https://www.kaggle.com/code/jhoward/getting-started-with-nlp-for-absolute-beginners

In [74]:
from datasets import Dataset,DatasetDict
ds = Dataset.from_pandas(train)
ds

Dataset({
    features: ['id', 'sentiment', 'review'],
    num_rows: 25000
})

In [72]:
model_nm = 'microsoft/deberta-v3-small'
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [78]:
tokz.tokenize(train.review[0])[:10]

['▁With',
 '▁all',
 '▁this',
 '▁stuff',
 '▁going',
 '▁down',
 '▁at',
 '▁the',
 '▁moment',
 '▁with']

In [75]:
def tok_func(x): return tokz(x["review"])

In [79]:
tok_ds = ds.map(tok_func, batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

In [81]:
tok_ds

Dataset({
    features: ['id', 'sentiment', 'review', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 25000
})

In [82]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'sentiment', 'review', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 18750
    })
    test: Dataset({
        features: ['id', 'sentiment', 'review', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6250
    })
})

In [88]:
dds['train']['sentiment'].count(1), dds['train']['sentiment'].count(0)

(9421, 9329)

In [89]:
dds['test']['sentiment'].count(1), dds['test']['sentiment'].count(0)

(3079, 3171)

In [94]:
dds['train'].rename_column('sentiment','labels')

Dataset({
    features: ['id', 'labels', 'review', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 18750
})

In [95]:
dds['test'].rename_column('sentiment','labels')

Dataset({
    features: ['id', 'labels', 'review', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6250
})

In [90]:
eval_ds = Dataset.from_pandas(test).map(tok_func, batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

In [91]:
eval_ds

Dataset({
    features: ['id', 'review', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 25000
})

In [92]:
from transformers import TrainingArguments,Trainer


In [93]:
bs = 128
epochs = 4
lr = 8e-5

args = TrainingArguments('hf-output', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')


In [97]:
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [98]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [102]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: id, sentiment, review. If id, sentiment, review are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 18750
  Num Epochs = 4
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 588


RuntimeError: CUDA out of memory. Tried to allocate 6.28 GiB (GPU 0; 15.90 GiB total capacity; 4.79 GiB already allocated; 5.81 GiB free; 9.39 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [101]:
## clears up cuda memory
import gc
gc.collect()

3291